Build a constellation of Planet Dove satellites

In [ ]:
import time
import numpy as np
import requests
from sedaro import SedaroApiClient
from sedaro_base_client.apis.tags import jobs_api

# Settings
API_KEY = '' 
SCENARIO_BRANCH_ID = 'NXVzrUNTFH9pNn8zTSH1F'
HOST = 'https://api.sedaro.com'
BASE_AGENT_ID = 'NT-KoZFSELKK8eomP3lkV'

### Delete satelllite with old TLEs

This is a cleanup step to remove any satellites that were created in a previous run of this notebook. This is not necessary if you are running this notebook for the first time.

In [ ]:
with SedaroApiClient(api_key=API_KEY, host=HOST) as sedaro_client:
    scenario = sedaro_client.get_branch(SCENARIO_BRANCH_ID)

    delete_ids=[]
    for agent_id in scenario.data['index']['Agent']:
        if agent_id != BASE_AGENT_ID and not (scenario.data['blocks'][agent_id]['peripheral']):
            delete_ids.append(agent_id)

    if not delete_ids: 
        print('No blocks were deleted')
    else:
        scenario.crud(delete=delete_ids)
        print(f'Deleted {len(delete_ids)} spacecraft digital twin agents.')

### Fetch Constellation TLEs

In [ ]:
# Load TLE file
url = 'https://ephemerides.planet-labs.com/planet_mc.tle'
tles = requests.get(url).text.splitlines()

### Create new agents with new TLEs

In [ ]:
with SedaroApiClient(api_key=API_KEY, host=HOST) as sedaro_client:
    scenario = sedaro_client.get_branch(SCENARIO_BRANCH_ID)
    base = scenario.Agent.get(BASE_AGENT_ID)

    def create_orbit(tle):
        return {**base.orbit.data, 'initialStateDefParams': {'tle': tle}}.pop('id')

    def create_agent(name, orbit_id):
        return {**base.data, 'orbit': orbit_id, 'name': name}.pop('id')

    n_sats = int(len(tles)/3)   #temporary limit of to the ammmount of sats in constellation due to issue with simulating all 218
    orbits=[create_orbit(tles[3*idx+1] + '\n' + tles[3*idx+2]) for idx in range(n_sats)]
    response = scenario.crud(blocks=orbits)
    print(f'Created {n_sats} orbits from TLEs.')

    orbit_ids = response['crud']['blocks']
    agents = [create_agent(tles[3*idx], orbit_ids[idx]) for idx in range(n_sats)]
    scenario.crud(blocks=agents)
    print(f'Created {n_sats} spacecraft digital twin agents.')